In [9]:
import missingno
import pandas as pd
import seaborn as sns
import numpy as np

sns.set_style("darkgrid")

In [10]:
df = pd.read_json("./tunann.jl", encoding="utf-8", lines=True)
df.dropna(how="all", inplace=True)

## Cleaning

### Price

In [11]:
df.price = pd.to_numeric(
    df.price.apply(lambda price_str: "".join(
        [i for i in str(price_str) if i.isdecimal()]))
)

### Created and edited dates

In [12]:
df.created = pd.to_datetime(df.created)
df.edited = pd.to_datetime(df.edited)

### Area

In [13]:
df.area = pd.to_numeric(
    df.area.apply(lambda area_str: "".join(
        [i for i in str(area_str) if i.isdecimal()]))
)

### Category

In [14]:
df["is_offer"] = df.category.str.split(">").apply(
    lambda x: str(x[0]).strip()) == "Offres"

df["estate_type"] = df.category.dropna().str.split(
    ">").apply(lambda x: x[2].strip())

df["transaction_type"] = df.category.dropna().str.split(
    ">").apply(lambda x: x[1].strip())

df.drop(columns="category", inplace=True)

### Location

In [15]:
df["governorate"] = df.location.str.split(
    ">").apply(lambda x: str(x[1]).strip())

df["delegation"] = df.location.str.split(
    ">").apply(lambda x: str(x[2]).strip())

df["municipality"] = df.location.str.split(
    ">").apply(lambda x: str(x[3]).strip())

df.drop(columns="location", inplace=True)

### Number of rooms

In [16]:
df["n_rooms"] = df.estate_type.replace({
    "Appart. 5 pièces+": "5",
    "Appart. 4 pièces": "4",
    "Appart. 3 pièces": "3",
    "Appart. 2 pièces": "2",
    "Appart. 1 pièce": "1"
})
df.estate_type.replace({
    "Appart. 5 pièces+": "Appartement",
    "Appart. 4 pièces": "Appartement",
    "Appart. 3 pièces": "Appartement",
    "Appart. 2 pièces": "Appartement",
    "Appart. 1 pièce": "Appartement"
}, inplace=True)
df.n_rooms.loc[-df.n_rooms.isin([str(i) for i in range(1, 6)])] = np.nan

/home/iyed/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
